In [6]:
# Import de Spark Session et SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.conf import SparkConf

conf = SparkConf()
conf.set("spark.log.level", "error") # Pour n'afficher que les erreurs
conf.set("spark.ui.showConsoleProgress", "false") # Pour ne pas afficher la progression des jobs Spark dans Python

# Définition d'un SparkContext
sc = SparkContext.getOrCreate(conf=conf)

# Définition d'une SparkSession
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Introduction au DataFrame") \
    .getOrCreate()

spark

In [7]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd = rdd.map(lambda x: x*2)
rdd.take(5)

[2, 4, 6, 8, 10]

In [8]:
sc.stop()

In [9]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("Cours de Spark").setMaster("local[*]")
sc = SparkContext(conf=conf)


Setting Spark log level to "ERROR".


In [10]:
RDD = sc.parallelize(range(10))


In [11]:
RDD

PythonRDD[1] at RDD at PythonRDD.scala:53

In [12]:
RDD.getNumPartitions()


2

In [13]:
RDD = sc.parallelize(range(50), 10)
print(RDD.glom().collect())


[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19], [20, 21, 22, 23, 24], [25, 26, 27, 28, 29], [30, 31, 32, 33, 34], [35, 36, 37, 38, 39], [40, 41, 42, 43, 44], [45, 46, 47, 48, 49]]


In [14]:
RDD = sc.parallelize(range(50), 100)

In [15]:
print(RDD.glom().collect())


[[], [0], [], [1], [], [2], [], [3], [], [4], [], [5], [], [6], [], [7], [], [8], [], [9], [], [10], [], [11], [], [12], [], [13], [], [14], [], [15], [], [16], [], [17], [], [18], [], [19], [], [20], [], [21], [], [22], [], [23], [], [24], [], [25], [], [26], [], [27], [], [28], [], [29], [], [30], [], [31], [], [32], [], [33], [], [34], [], [35], [], [36], [], [37], [], [38], [], [39], [], [40], [], [41], [], [42], [], [43], [], [44], [], [45], [], [46], [], [47], [], [48], [], [49]]


In [16]:
RDD = sc.parallelize(range(3), 2)
print(RDD.glom().collect())


[[0], [1, 2]]


In [17]:
import numpy as np
# On créé un tableau de dimension 2

array = np.array([[1,2],
                  [3,4]])

# On vérifie si sa version de dimension 1 est égale au résultat de la méthode flatten

print((np.array([1,2,3,4]) == array.flatten()).all())


True


In [18]:
def r(f, l):
    k=1
    res=l[0]
    while k < len(l):
        res=f(l[k],res)
        k+=1
    if k==1:
        raise ValueError
    return res

In [19]:
r(lambda x,y:x+y,range(10))

45

In [20]:
def fr(f,L):
    return r(f,[r(f, l) for l in L])

In [21]:
fr(lambda x,y:x+y,[[0,1,2], [3,4,5], [6,7,8,9]])

45

In [22]:
# correction
def Reduce(f, L):
    while len(L) > 1:
        L[1] = f(L[0], L[1])
        L.pop(0)
    return(L)

L = list(range(10))
print(Reduce(lambda x,y: x+y, L))


[45]


In [23]:
# correction
def reduce(f, L):
    while len(L) > 1:
        L[1] = f(L[0], L[1])
        L.pop(0)
    return(L)

def reduceGlobal(f, L):
    for i, element in enumerate(L):
        reduce(f, element)
        L[i] = element[0]
    return(reduce(f, L))

L = [[0,1,2], [3,4,5], [6,7,8,9]]
print(reduceGlobal(lambda x,y: x+y, L))


[45]


In [24]:
count = 0
rdd = sc.parallelize([1,2,3,4,5])

def add_count(x):
    global count
    count = count + x

rdd.foreach(add_count)
print(count)


0


In [25]:
count = 0
rdd = sc.parallelize(range(50))

def add_count(x):
    global count
    count = count + x

rdd.foreach(add_count)
print(count)


0


In [26]:
RDD = sc.parallelize(range(50), 3)


print(RDD.glom().collect())

def partition_sum(partition):
    return([sum(partition)])
print('---')
print(RDD.mapPartitions(partition_sum).collect())


[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32], [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]]
---
[120, 408, 697]


In [27]:
RDD = sc.parallelize(range(50))
RDD.filter(lambda x:x%2==0).collect()

[0,
 2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 22,
 24,
 26,
 28,
 30,
 32,
 34,
 36,
 38,
 40,
 42,
 44,
 46,
 48]

In [28]:
RDD.filter(lambda x:x%2==0).sum()

600

In [29]:
RDD.filter(lambda x:x%2!=0).count()

25

In [30]:
RDD = sc.parallelize(range(50))

def odd_key(value):
    if value%2 == 0:
        return(0, value)
    else:
        return(1, value)

RDD1 = RDD.map(odd_key)
print(RDD1.reduceByKey(lambda x, y: x+y).collect())

[(0, 600), (1, 625)]


In [31]:
RDD.filter(lambda x:x%2!=0).sortBy(lambda x:x).collect()

[1,
 3,
 5,
 7,
 9,
 11,
 13,
 15,
 17,
 19,
 21,
 23,
 25,
 27,
 29,
 31,
 33,
 35,
 37,
 39,
 41,
 43,
 45,
 47,
 49]

In [32]:
RDD.filter(lambda x:x%2!=0).sortBy(lambda x:x).collect()

[1,
 3,
 5,
 7,
 9,
 11,
 13,
 15,
 17,
 19,
 21,
 23,
 25,
 27,
 29,
 31,
 33,
 35,
 37,
 39,
 41,
 43,
 45,
 47,
 49]

In [33]:
RDD = sc.parallelize(range(50))

def odd_key(value):
    if value%2 == 0:
        return(0, value)
    else:
        return(1, value)

RDD1 = RDD.map(odd_key)
print(RDD1.sortByKey().collect())


[(0, 0), (0, 2), (0, 4), (0, 6), (0, 8), (0, 10), (0, 12), (0, 14), (0, 16), (0, 18), (0, 20), (0, 22), (0, 24), (0, 26), (0, 28), (0, 30), (0, 32), (0, 34), (0, 36), (0, 38), (0, 40), (0, 42), (0, 44), (0, 46), (0, 48), (1, 1), (1, 3), (1, 5), (1, 7), (1, 9), (1, 11), (1, 13), (1, 15), (1, 17), (1, 19), (1, 21), (1, 23), (1, 25), (1, 27), (1, 29), (1, 31), (1, 33), (1, 35), (1, 37), (1, 39), (1, 41), (1, 43), (1, 45), (1, 47), (1, 49)]


In [34]:
RDD = sc.parallelize(range(50))
RDD1 = RDD.map(lambda x: (x%2, x))


def partitioning(key):
    if key==0:
        return(0)
    else:
        return(1)


RDD1 = RDD1.partitionBy(2, partitioning)
print(RDD1.glom().collect())


[[(0, 0), (0, 2), (0, 4), (0, 6), (0, 8), (0, 10), (0, 12), (0, 14), (0, 16), (0, 18), (0, 20), (0, 22), (0, 24), (0, 26), (0, 28), (0, 30), (0, 32), (0, 34), (0, 36), (0, 38), (0, 40), (0, 42), (0, 44), (0, 46), (0, 48)], [(1, 1), (1, 3), (1, 5), (1, 7), (1, 9), (1, 11), (1, 13), (1, 15), (1, 17), (1, 19), (1, 21), (1, 23), (1, 25), (1, 27), (1, 29), (1, 31), (1, 33), (1, 35), (1, 37), (1, 39), (1, 41), (1, 43), (1, 45), (1, 47), (1, 49)]]


In [35]:
RDD = sc.textFile("shakespeare.txt")
RDD_clean = RDD.flatMap(lambda x: x.replace('\'', ' ').lower().split())


In [36]:
RDD_clean.collect()

['a',
 'midsummer-night',
 's',
 'dream',
 'now',
 ',',
 'fair',
 'hippolyta',
 ',',
 'our',
 'nuptial',
 'hour',
 'draws',
 'on',
 'apace',
 ':',
 'four',
 'happy',
 'days',
 'bring',
 'in',
 'another',
 'moon',
 ';',
 'but',
 'o',
 '!',
 'methinks',
 'how',
 'slow',
 'this',
 'old',
 'moon',
 'wanes',
 ';',
 'she',
 'lingers',
 'my',
 'desires',
 ',',
 'like',
 'to',
 'a',
 'step',
 'dame',
 ',',
 'or',
 'a',
 'dowager',
 'long',
 'withering',
 'out',
 'a',
 'young',
 'man',
 's',
 'revenue',
 '.',
 'four',
 'days',
 'will',
 'quickly',
 'steep',
 'themselves',
 'in',
 'night',
 ';',
 'four',
 'nights',
 'will',
 'quickly',
 'dream',
 'away',
 'the',
 'time',
 ';',
 'and',
 'then',
 'the',
 'moon',
 ',',
 'like',
 'to',
 'a',
 'silver',
 'bow',
 'new-bent',
 'in',
 'heaven',
 ',',
 'shall',
 'behold',
 'the',
 'night',
 'of',
 'our',
 'solemnities',
 '.',
 'go',
 ',',
 'philostrate',
 ',',
 'stir',
 'up',
 'the',
 'athenian',
 'youth',
 'to',
 'merriments',
 ';',
 'awake',
 'the',
 '

In [37]:
RDD_count = RDD_clean.map(lambda x: (x, 1))
print(RDD_count.reduceByKey(lambda x,y : x+y).sortBy(lambda x: x[1], False).collect())


[(',', 5229), ('.', 2465), ('the', 1708), ('i', 1685), ('and', 1540), ('to', 1121), ('you', 1080), (';', 1076), ('of', 1017), ('a', 980), (':', 795), ('that', 771), ('in', 768), ('my', 761), ('is', 695), ('?', 620), ('not', 616), ('it', 551), ('me', 542), ('he', 501), ('for', 483), ('!', 480), ('your', 468), ('with', 461), ('s', 454), ('but', 447), ('his', 436), ('be', 432), ('this', 417), ('have', 407), ('d', 381), ('will', 378), ('as', 365), ('so', 360), ('him', 343), ('thou', 338), ('her', 336), ('do', 296), ('what', 287), ('no', 285), ('if', 255), ('shall', 236), ('love', 235), ('all', 235), ('are', 233), ('we', 232), ('by', 219), ('thee', 209), ('thy', 208), ('good', 202), ('she', 196), ('more', 188), ('on', 186), ('our', 185), ('am', 176), ('would', 174), ('was', 174), ('man', 171), ('o', 169), ('here', 167), ('well', 163), ('or', 160), ('which', 160), ('come', 159), ('now', 159), ('from', 158), ('when', 156), ('at', 150), ('there', 146), ('sir', 143), ('lord', 142), ('ll', 140),

# 3

In [38]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("Cours de Spark")\
        .master("local[*]")\
        .getOrCreate()
sc = spark.sparkContext

In [39]:
L = [[0, "Alexandre", 28, "Chatou", "Décembre"],\
     [1, "Damien", 24, "Courbevoie", "Novembre"],\
     [2, "Dimitri", 25, "Sibiu", "Octobre"],\
     [3, "Sacha", 32, "Poissy", "Septembre"],\
     [4, "Victor", 21, "Montrouge", "Juin"]]

rdd = sc.parallelize(L)

In [40]:
df = rdd.toDF()


In [41]:
df.printSchema()


root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: long (nullable = true)
 |-- _4: string (nullable = true)
 |-- _5: string (nullable = true)



In [42]:
df.show()


+---+---------+---+----------+---------+
| _1|       _2| _3|        _4|       _5|
+---+---------+---+----------+---------+
|  0|Alexandre| 28|    Chatou| Décembre|
|  1|   Damien| 24|Courbevoie| Novembre|
|  2|  Dimitri| 25|     Sibiu|  Octobre|
|  3|    Sacha| 32|    Poissy|Septembre|
|  4|   Victor| 21| Montrouge|     Juin|
+---+---------+---+----------+---------+



In [43]:
df = rdd.toDF(["", "Prénom", "Âge", "Ville", "Cohorte"])
df.printSchema()
df.show()

root
 |-- : long (nullable = true)
 |-- Prénom: string (nullable = true)
 |-- Âge: long (nullable = true)
 |-- Ville: string (nullable = true)
 |-- Cohorte: string (nullable = true)

+---+---------+---+----------+---------+
|   |   Prénom|Âge|     Ville|  Cohorte|
+---+---------+---+----------+---------+
|  0|Alexandre| 28|    Chatou| Décembre|
|  1|   Damien| 24|Courbevoie| Novembre|
|  2|  Dimitri| 25|     Sibiu|  Octobre|
|  3|    Sacha| 32|    Poissy|Septembre|
|  4|   Victor| 21| Montrouge|     Juin|
+---+---------+---+----------+---------+



In [44]:
from pyspark.sql.types import *

schema = StructType([StructField("", IntegerType(), False),\
                     StructField("Prénom", StringType()),\
                     StructField("Âge", IntegerType()),\
                     StructField("Ville", StringType()),\
                     StructField("Cohorte", StringType())])


In [45]:
schema

StructType([StructField('', IntegerType(), False), StructField('Prénom', StringType(), True), StructField('Âge', IntegerType(), True), StructField('Ville', StringType(), True), StructField('Cohorte', StringType(), True)])

In [46]:
df = rdd.toDF(schema)
df.printSchema()
df.show()


root
 |-- : integer (nullable = false)
 |-- Prénom: string (nullable = true)
 |-- Âge: integer (nullable = true)
 |-- Ville: string (nullable = true)
 |-- Cohorte: string (nullable = true)

+---+---------+---+----------+---------+
|   |   Prénom|Âge|     Ville|  Cohorte|
+---+---------+---+----------+---------+
|  0|Alexandre| 28|    Chatou| Décembre|
|  1|   Damien| 24|Courbevoie| Novembre|
|  2|  Dimitri| 25|     Sibiu|  Octobre|
|  3|    Sacha| 32|    Poissy|Septembre|
|  4|   Victor| 21| Montrouge|     Juin|
+---+---------+---+----------+---------+



In [47]:
df = spark.read.csv("csv_comma.csv")

In [48]:
df.printSchema()
df.show()


root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)

+---------+---+----------+---------+
|      _c0|_c1|       _c2|      _c3|
+---------+---+----------+---------+
|   Prénom|Âge|     Ville|  Cohorte|
|Alexandre| 28|    Chatou| Décembre|
|   Damien| 24|Courbevoie| Novembre|
|  Dimitri| 25|     Sibiu|  Octobre|
|    Sacha| 32|    Poissy|Septembre|
|   Victor| 21| Montrouge|     Juin|
+---------+---+----------+---------+



In [49]:
schema = StructType([StructField("Prénom", StringType()),\
                     StructField("Âge", IntegerType()),\
                     StructField("Ville", StringType()),\
                     StructField("Cohorte", StringType())])

df = spark.read.schema(schema)\
               .option("delimiter", ",")\
               .option("header", True)\
               .csv("csv_comma.csv")
df.printSchema()
df.show()


root
 |-- Prénom: string (nullable = true)
 |-- Âge: integer (nullable = true)
 |-- Ville: string (nullable = true)
 |-- Cohorte: string (nullable = true)

+---------+---+----------+---------+
|   Prénom|Âge|     Ville|  Cohorte|
+---------+---+----------+---------+
|Alexandre| 28|    Chatou| Décembre|
|   Damien| 24|Courbevoie| Novembre|
|  Dimitri| 25|     Sibiu|  Octobre|
|    Sacha| 32|    Poissy|Septembre|
|   Victor| 21| Montrouge|     Juin|
+---------+---+----------+---------+



In [50]:
df = spark.read.option("multiLine",True).json("json_bracket.json")


In [51]:
df.printSchema()
df.show()

root
 |-- Cohorte: string (nullable = true)
 |-- Prénom: string (nullable = true)
 |-- Ville: string (nullable = true)
 |-- Âge: long (nullable = true)

+---------+---------+----------+---+
|  Cohorte|   Prénom|     Ville|Âge|
+---------+---------+----------+---+
| Décembre|Alexandre|    Chatou| 28|
| Novembre|   Damien|Courbevoie| 24|
|  Octobre|  Dimitri|     Sibiu| 25|
|Septembre|    Sacha|    Poissy| 32|
|     Juin|   Victor| Montrouge| 21|
+---------+---------+----------+---+



In [52]:
df = spark.read.schema(schema).option("multiLine",True).json("json_bracket.json")
df.show()
df.printSchema()


+---------+---+----------+---------+
|   Prénom|Âge|     Ville|  Cohorte|
+---------+---+----------+---------+
|Alexandre| 28|    Chatou| Décembre|
|   Damien| 24|Courbevoie| Novembre|
|  Dimitri| 25|     Sibiu|  Octobre|
|    Sacha| 32|    Poissy|Septembre|
|   Victor| 21| Montrouge|     Juin|
+---------+---+----------+---------+

root
 |-- Prénom: string (nullable = true)
 |-- Âge: integer (nullable = true)
 |-- Ville: string (nullable = true)
 |-- Cohorte: string (nullable = true)



In [53]:
df = spark.read.option("header", True)\
               .option("inferSchema", True)\
               .option("escape", "\"")\
               .csv("gps_app.csv")

df.show(5)
df.printSchema()


+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Ske

In [54]:
print(df.head(5),end='\n\n')
#print(df.take(5))
print(df.tail(5),end='\n\n')
print(df.first(),end='\n\n')
#print(df.head(5)[0])
#print(df.take(5)[0])


[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'), Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'), Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'), Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating=4.5, Reviews='215644', Size='25M', 

In [55]:
df.summary("min","13%","mean","max").show()


+-------+--------------------+--------+------+------------------+------------------+--------+----+--------+---------------+------+-----------------+-------------+------------------+
|summary|                 App|Category|Rating|           Reviews|              Size|Installs|Type|   Price| Content Rating|Genres|     Last Updated|  Current Ver|       Android Ver|
+-------+--------------------+--------+------+------------------+------------------+--------+----+--------+---------------+------+-----------------+-------------+------------------+
|    min|"i DT" Fútbol. To...|     1.9|   1.0|                 0|            1,000+|       0|   0|   $0.99|Adults only 18+|Action|           1.0.19|      0.0.0.2|        1.0 and up|
|    13%|                NULL|     1.9|   3.8|               5.0|              NULL|     0.0| NaN|     0.0|           NULL|  NULL|             NULL|          1.0|               NaN|
|   mean|                NULL|     1.9|   NaN|444152.89603321033|              NULL|     0

In [56]:
df.summary().show()


+-------+--------------------+--------+------+------------------+------------------+--------+-----+--------+---------------+------+-----------------+-------------+------------------+
|summary|                 App|Category|Rating|           Reviews|              Size|Installs| Type|   Price| Content Rating|Genres|     Last Updated|  Current Ver|       Android Ver|
+-------+--------------------+--------+------+------------------+------------------+--------+-----+--------+---------------+------+-----------------+-------------+------------------+
|  count|               10841|   10841| 10841|             10841|             10841|   10841|10841|   10841|          10840| 10841|            10841|        10840|             10840|
|   mean|                NULL|     1.9|   NaN|444152.89603321033|              NULL|     0.0|  NaN|     0.0|           NULL|  NULL|             NULL|          NaN|               NaN|
| stddev|                NULL|    NULL|   NaN|2927760.6038856646|              NULL| 

In [57]:
df.describe(["Reviews"]).show()


+-------+------------------+
|summary|           Reviews|
+-------+------------------+
|  count|             10841|
|   mean|444152.89603321033|
| stddev|2927760.6038856646|
|    min|                 0|
|    max|              9992|
+-------+------------------+



In [58]:
df.select("App").show()


+--------------------+
|                 App|
+--------------------+
|Photo Editor & Ca...|
| Coloring book moana|
|U Launcher Lite –...|
|Sketch - Draw & P...|
|Pixel Draw - Numb...|
|Paper flowers ins...|
|Smoke Effect Phot...|
|    Infinite Painter|
|Garden Coloring Book|
|Kids Paint Free -...|
|Text on Photo - F...|
|Name Art Photo Ed...|
|Tattoo Name On My...|
|Mandala Coloring ...|
|3D Color Pixel by...|
|Learn To Draw Kaw...|
|Photo Designer - ...|
|350 Diy Room Deco...|
|FlipaClip - Carto...|
|        ibis Paint X|
+--------------------+
only showing top 20 rows



In [59]:
df.select(df["App"]).show()


+--------------------+
|                 App|
+--------------------+
|Photo Editor & Ca...|
| Coloring book moana|
|U Launcher Lite –...|
|Sketch - Draw & P...|
|Pixel Draw - Numb...|
|Paper flowers ins...|
|Smoke Effect Phot...|
|    Infinite Painter|
|Garden Coloring Book|
|Kids Paint Free -...|
|Text on Photo - F...|
|Name Art Photo Ed...|
|Tattoo Name On My...|
|Mandala Coloring ...|
|3D Color Pixel by...|
|Learn To Draw Kaw...|
|Photo Designer - ...|
|350 Diy Room Deco...|
|FlipaClip - Carto...|
|        ibis Paint X|
+--------------------+
only showing top 20 rows



In [60]:
from pyspark.sql.functions import *
df.select(col("App")).show()


+--------------------+
|                 App|
+--------------------+
|Photo Editor & Ca...|
| Coloring book moana|
|U Launcher Lite –...|
|Sketch - Draw & P...|
|Pixel Draw - Numb...|
|Paper flowers ins...|
|Smoke Effect Phot...|
|    Infinite Painter|
|Garden Coloring Book|
|Kids Paint Free -...|
|Text on Photo - F...|
|Name Art Photo Ed...|
|Tattoo Name On My...|
|Mandala Coloring ...|
|3D Color Pixel by...|
|Learn To Draw Kaw...|
|Photo Designer - ...|
|350 Diy Room Deco...|
|FlipaClip - Carto...|
|        ibis Paint X|
+--------------------+
only showing top 20 rows



In [61]:
df.select(4 * col("Rating")).show()

+------------+
|(Rating * 4)|
+------------+
|        16.4|
|        15.6|
|        18.8|
|        18.0|
|        17.2|
|        17.6|
|        15.2|
|        16.4|
|        17.6|
|        18.8|
|        17.6|
|        17.6|
|        16.8|
|        18.4|
|        17.6|
|        12.8|
|        18.8|
|        18.0|
|        17.2|
|        18.4|
+------------+
only showing top 20 rows



In [62]:
df_20 = df.withColumn("Rating/20", 4 * col("Rating"))
df_20.show()


+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+---------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|Rating/20|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+---------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|     16.4|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|     15.6|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & D

In [63]:
df_renamed = df_20.withColumnRenamed("Reviews", "Nbr of Reviews")
df_renamed.show()


+--------------------+--------------+------+--------------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+---------+
|                 App|      Category|Rating|Nbr of Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|Rating/20|
+--------------------+--------------+------+--------------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+---------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|           159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|     16.4|
| Coloring book moana|ART_AND_DESIGN|   3.9|           967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|     15.6|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|         87510|8.7M| 5,000,000+

In [64]:
df_renamed.where(col("Rating/20") < 15).show()


+--------------------+-------------------+------+--------------+------------------+-----------+----+-----+--------------+-----------------+------------------+--------------------+------------------+---------+
|                 App|           Category|Rating|Nbr of Reviews|              Size|   Installs|Type|Price|Content Rating|           Genres|      Last Updated|         Current Ver|       Android Ver|Rating/20|
+--------------------+-------------------+------+--------------+------------------+-----------+----+-----+--------------+-----------------+------------------+--------------------+------------------+---------+
|Learn To Draw Kaw...|     ART_AND_DESIGN|   3.2|            55|              2.7M|     5,000+|Free|    0|      Everyone|     Art & Design|      June 6, 2018|                 NaN|        4.2 and up|     12.8|
|SK Enca Direct Ma...|  AUTO_AND_VEHICLES|   3.6|          1379|               16M|   500,000+|Free|    0|      Everyone|  Auto & Vehicles|    August 2, 2018|      

In [65]:


df_renamed.where((col("Rating/20") > 15) | (col("Rating/20") < 5)).show()

+--------------------+--------------+------+--------------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+---------+
|                 App|      Category|Rating|Nbr of Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|Rating/20|
+--------------------+--------------+------+--------------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+---------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|           159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|     16.4|
| Coloring book moana|ART_AND_DESIGN|   3.9|           967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|     15.6|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|         87510|8.7M| 5,000,000+

In [66]:

df_renamed.select(when(col("Rating/20") >= 18, "excellente").\
                  when(col("Rating/20") >= 16, "très bien").\
                  when(col("Rating/20") >= 14, "bien").\
                  when(col("Rating/20") >= 12, "assez bien").\
                  when(col("Rating/20") >= 10, "passable").\
                  otherwise("mauvaise").alias("appréciation"), "Rating/20").show()


+------------+---------+
|appréciation|Rating/20|
+------------+---------+
|   très bien|     16.4|
|        bien|     15.6|
|  excellente|     18.8|
|  excellente|     18.0|
|   très bien|     17.2|
|   très bien|     17.6|
|        bien|     15.2|
|   très bien|     16.4|
|   très bien|     17.6|
|  excellente|     18.8|
|   très bien|     17.6|
|   très bien|     17.6|
|   très bien|     16.8|
|  excellente|     18.4|
|   très bien|     17.6|
|  assez bien|     12.8|
|  excellente|     18.8|
|  excellente|     18.0|
|   très bien|     17.2|
|  excellente|     18.4|
+------------+---------+
only showing top 20 rows



In [67]:
df_renamed.select(when(col("Rating/20") >= 10, "passable").\
                  when(col("Rating/20") >= 12, "assez bien").\
                  when(col("Rating/20") >= 14, "bien").\
                  when(col("Rating/20") >= 16, "très bien").\
                  when(col("Rating/20") >= 18, "excellente").\
                  otherwise("mauvaise").alias("appréciation"), "Rating/20").show()


+------------+---------+
|appréciation|Rating/20|
+------------+---------+
|    passable|     16.4|
|    passable|     15.6|
|    passable|     18.8|
|    passable|     18.0|
|    passable|     17.2|
|    passable|     17.6|
|    passable|     15.2|
|    passable|     16.4|
|    passable|     17.6|
|    passable|     18.8|
|    passable|     17.6|
|    passable|     17.6|
|    passable|     16.8|
|    passable|     18.4|
|    passable|     17.6|
|    passable|     12.8|
|    passable|     18.8|
|    passable|     18.0|
|    passable|     17.2|
|    passable|     18.4|
+------------+---------+
only showing top 20 rows



In [68]:
df_used = df_renamed.withColumn("used", when(col("Nbr of Reviews") >= 10000, True).\
                                        otherwise(False))
df_used.show()


+--------------------+--------------+------+--------------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+---------+-----+
|                 App|      Category|Rating|Nbr of Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|Rating/20| used|
+--------------------+--------------+------+--------------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+---------+-----+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|           159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|     16.4|false|
| Coloring book moana|ART_AND_DESIGN|   3.9|           967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|     15.6|false|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7| 

In [69]:
df_renamed.select((col("Nbr of Reviews") >= 10000).alias("used"), "Nbr of Reviews").show()


+-----+--------------+
| used|Nbr of Reviews|
+-----+--------------+
|false|           159|
|false|           967|
| true|         87510|
| true|        215644|
|false|           967|
|false|           167|
|false|           178|
| true|         36815|
| true|         13791|
|false|           121|
| true|         13880|
|false|          8788|
| true|         44829|
|false|          4326|
|false|          1518|
|false|            55|
|false|          3632|
|false|            27|
| true|        194216|
| true|        224399|
+-----+--------------+
only showing top 20 rows



In [70]:
count_original = df_used.count()
count_distinct = df_used.distinct().count()
count_duplicates = count_original - count_distinct
print(count_original, count_distinct, count_duplicates)


10841 10358 483


In [71]:
df_unique = df_used.dropDuplicates(['App'])
count_app_distinct = df_unique.count()
print(count_original, count_distinct, count_app_distinct)


10841 10358 9660


In [72]:
df_used.columns

['App',
 'Category',
 'Rating',
 'Nbr of Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver',
 'Rating/20',
 'used']

In [73]:
import numpy as np
from pyspark.sql.functions import isnan, isnull
from pyspark.sql.types import BooleanType

def getMissingValues(dataframe):
  count = dataframe.count()
  columns = dataframe.columns
  nan_count = []
  # we can't check for nan in a boolean type column
  for column in columns:
    if dataframe.schema[column].dataType == BooleanType():
      nan_count.append(0)
    else:
      nan_count.append(dataframe.where(isnan(col(column))).count())
  null_count = [dataframe.where(isnull(col(column))).count() for column in columns]
  return([count, columns, nan_count, null_count])

def missingTable(stats):
  count, columns, nan_count, null_count = stats
  count = str(count)
  nan_count = [str(element) for element in nan_count]
  null_count = [str(element) for element in null_count]
  max_init = np.max([len(str(count)), 10])
  line1 = "+" + max_init*"-" + "+"
  line2 = "|" + (max_init-len(count))*" " + count + "|"
  line3 = "|" + (max_init-9)*" " + "nan count|"
  line4 = "|" + (max_init-10)*" " + "null count|"
  for i in range(len(columns)):
    max_column = np.max([len(columns[i]),\
                        len(nan_count[i]),\
                        len(null_count[i])])
    line1 += max_column*"-" + "+"
    line2 += (max_column - len(columns[i]))*" " + columns[i] + "|"
    line3 += (max_column - len(nan_count[i]))*" " + nan_count[i] + "|"
    line4 += (max_column - len(null_count[i]))*" " + null_count[i] + "|"
  lines = f"{line1}\n{line2}\n{line1}\n{line3}\n{line4}\n{line1}"
  print(lines)

missingTable(getMissingValues(df_unique))

+----------+---+--------+------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+
|      9660|App|Category|Rating|Nbr of Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|Rating/20|used|
+----------+---+--------+------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+
| nan count|  0|       0|  1463|             0|   0|       0|   1|    0|             0|     0|           0|          7|          2|     1463|   0|
|null count|  0|       0|     0|             0|   0|       0|   0|    0|             1|     0|           0|          1|          1|        0|   0|
+----------+---+--------+------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+


In [74]:
df_clean = df_unique.drop("Rating")\
                    .dropna(subset=["Type", "Content Rating", "Android Ver"])

missingTable(getMissingValues(df_clean))


+----------+---+--------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+
|      9659|App|Category|Nbr of Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|Rating/20|used|
+----------+---+--------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+
| nan count|  0|       0|             0|   0|       0|   1|    0|             0|     0|           0|          7|          2|     1463|   0|
|null count|  0|       0|             0|   0|       0|   0|    0|             0|     0|           0|          1|          0|        0|   0|
+----------+---+--------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+


In [75]:
df_clean = df_clean.filter(~isnan("Type") & ~isnan("Android Ver"))

missingTable(getMissingValues(df_clean))


+----------+---+--------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+
|      9656|App|Category|Nbr of Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|Rating/20|used|
+----------+---+--------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+
| nan count|  0|       0|             0|   0|       0|   0|    0|             0|     0|           0|          7|          0|     1462|   0|
|null count|  0|       0|             0|   0|       0|   0|    0|             0|     0|           0|          1|          0|        0|   0|
+----------+---+--------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+


In [76]:
df_clean.filter(~isnan("Rating/20")).select(avg("Rating/20")).collect()

[Row(avg(Rating/20)=16.692701977056327)]

In [77]:
# We start by computing the average, be careful not to keep missing values

rating_average_df = df_clean.filter(~isnan("Rating/20")).select(avg("Rating/20"))

# Then we extract the corresponding float by using the head method

rating_average = rating_average_df.head()["avg(Rating/20)"]

# Now we can fill the missing values of the Rating/20 column

df_nomiss = df_clean.fillna({"Rating/20": rating_average})

# We check to see if we have remaining missing values.

missingTable(getMissingValues(df_nomiss))

+----------+---+--------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+
|      9656|App|Category|Nbr of Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|Rating/20|used|
+----------+---+--------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+
| nan count|  0|       0|             0|   0|       0|   0|    0|             0|     0|           0|          7|          0|        0|   0|
|null count|  0|       0|             0|   0|       0|   0|    0|             0|     0|           0|          1|          0|        0|   0|
+----------+---+--------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+


In [78]:
grouped_ver = df_nomiss.groupBy("Current Ver").count().sort("count", ascending=False)
mode_ver = grouped_ver.head()["Current Ver"]
print(mode_ver)

df_final = df_nomiss.withColumn("Current Ver", when(isnan("Current Ver") | isnull("Current Ver"), mode_ver)\
                                              .otherwise(col("Current Ver")))

missingTable(getMissingValues(df_final))

Varies with device
+----------+---+--------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+
|      9656|App|Category|Nbr of Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|Rating/20|used|
+----------+---+--------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+
| nan count|  0|       0|             0|   0|       0|   0|    0|             0|     0|           0|          0|          0|        0|   0|
|null count|  0|       0|             0|   0|       0|   0|    0|             0|     0|           0|          0|          0|        0|   0|
+----------+---+--------+--------------+----+--------+----+-----+--------------+------+------------+-----------+-----------+---------+----+


In [45]:
df_final.groupBy("Category").agg({"Genres": "mode", "Rating/20": "mean"}).show()


+-------------------+------------------+--------------------+
|           Category|    avg(Rating/20)|        mode(Genres)|
+-------------------+------------------+--------------------+
|             EVENTS|17.430645899438602|              Events|
|             SPORTS|16.830232703103583|              Sports|
|             COMICS|16.724739356323443|              Comics|
|            WEATHER| 16.94745460556195|             Weather|
|      VIDEO_PLAYERS|16.225708770894755|Video Players & E...|
|  AUTO_AND_VEHICLES| 16.75191086734913|     Auto & Vehicles|
|          PARENTING| 17.11545032950939|           Parenting|
|      ENTERTAINMENT| 16.54117647058823|       Entertainment|
|    PERSONALIZATION|17.193665118209623|     Personalization|
| HEALTH_AND_FITNESS| 16.92944057982804|    Health & Fitness|
|   TRAVEL_AND_LOCAL|16.338659649615547|      Travel & Local|
|BOOKS_AND_REFERENCE|17.215825246774724|   Books & Reference|
|     FOOD_AND_DRINK|16.689898532026906|        Food & Drink|
|       

In [47]:
df_final.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Nbr of Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)
 |-- Rating/20: double (nullable = false)
 |-- used: boolean (nullable = false)



In [48]:
df_user = spark.read.option("header", True)\
                    .option("inferSchema", True)\
                    .option("escape", "\"")\
                    .csv("gps_user.csv")


In [49]:
df_user.show()

+--------------------+--------------------+---------+-------------------+----------------------+
|                 App|   Translated_Review|Sentiment| Sentiment_Polarity|Sentiment_Subjectivity|
+--------------------+--------------------+---------+-------------------+----------------------+
|10 Best Foods for...|I like eat delici...| Positive|                1.0|    0.5333333333333333|
|10 Best Foods for...|This help eating ...| Positive|               0.25|   0.28846153846153844|
|10 Best Foods for...|                 nan|      nan|                nan|                   nan|
|10 Best Foods for...|Works great espec...| Positive|                0.4|                 0.875|
|10 Best Foods for...|        Best idea us| Positive|                1.0|                   0.3|
|10 Best Foods for...|            Best way| Positive|                1.0|                   0.3|
|10 Best Foods for...|             Amazing| Positive| 0.6000000000000001|                   0.9|
|10 Best Foods for...|        

In [50]:
df_user.printSchema()

root
 |-- App: string (nullable = true)
 |-- Translated_Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- Sentiment_Polarity: string (nullable = true)
 |-- Sentiment_Subjectivity: string (nullable = true)



In [51]:
df_joined = df_final.join(df_user, "App")
df_joined.filter(~isnan("Sentiment_Polarity") & ~isnull("Sentiment_Polarity"))\
         .select("App", col("Sentiment_Polarity").cast("double"))\
         .groupBy("App")\
         .avg("Sentiment_Polarity")\
         .sort("avg(Sentiment_Polarity)", ascending=False)\
         .show()


+--------------------+-----------------------+
|                 App|avg(Sentiment_Polarity)|
+--------------------+-----------------------+
|            HomeWork|                    1.0|
|       Google Slides|     0.9333333333333332|
|Daily Workouts - ...|                    0.8|
|Bed Time Fan - Wh...|                0.78125|
|Cameringo Lite. F...|     0.7702690972222221|
|       Google Primer|                   0.75|
|        GPS Map Free|                    0.7|
|GPS Speedometer a...|                 0.6875|
|Best Ovulation Tr...|              0.5953125|
|3D Live Neon Weed...|     0.5681818181818181|
|    Cookbook Recipes|     0.5654370380932882|
|CM Flashlight (Co...|                0.55625|
|Goldstar: Live Ev...|     0.5492664141414142|
|Hipmunk Hotels & ...|     0.5463015873015874|
|850 Sports News D...|     0.5428597480561768|
|       C++ Tutorials|     0.5309259259259259|
|Haystack TV: Loca...|                 0.5295|
|All Football GO- ...|     0.5265151515151514|
|Home Workout

In [80]:
df_final.createOrReplaceTempView("APPS")

In [81]:
spark.sql("SELECT count(*) as freq, `Content Rating` FROM APPS GROUP BY `Content Rating` ORDER BY freq DESC").show()


+----+---------------+
|freq| Content Rating|
+----+---------------+
|7901|       Everyone|
|1036|           Teen|
| 393|     Mature 17+|
| 321|   Everyone 10+|
|   3|Adults only 18+|
|   2|        Unrated|
+----+---------------+



In [82]:
spark.sql("SELECT * FROM APPS WHERE `Content Rating`='Adults only 18+'").show()


+--------------------+--------+--------------+------------------+----------+----+-----+---------------+------+--------------+------------------+------------------+---------+-----+
|                 App|Category|Nbr of Reviews|              Size|  Installs|Type|Price| Content Rating|Genres|  Last Updated|       Current Ver|       Android Ver|Rating/20| used|
+--------------------+--------+--------------+------------------+----------+----+-----+---------------+------+--------------+------------------+------------------+---------+-----+
|DraftKings - Dail...|  SPORTS|         50017|               41M|1,000,000+|Free|    0|Adults only 18+|Sports| July 24, 2018|          3.21.324|        4.4 and up|     18.0| true|
|         Manga Books|  COMICS|          7326|Varies with device|  500,000+|Free|    0|Adults only 18+|Comics|August 3, 2018|Varies with device|Varies with device|     15.2|false|
|Manga Master - Be...|  COMICS|         24005|              4.9M|  500,000+|Free|    0|Adults only 1

In [87]:
req="""
SELECT count(*) as freq, Category FROM APPS
WHERE `Rating/20`>=15
GROUP BY `Category`
HAVING freq >= 400
"""
spark.sql(req).show()

+----+--------+
|freq|Category|
+----+--------+
|1590|  FAMILY|
| 886|    GAME|
| 662|   TOOLS|
+----+--------+



In [88]:
spark.sql("""
  SELECT count(*) as `good apps`, Category
  FROM APPS 
  WHERE `Rating/20` > 15
  GROUP BY CATEGORY
  HAVING `good apps` > 400
  ORDER BY `good apps` DESC
  """).show()


+---------+--------+
|good apps|Category|
+---------+--------+
|     1590|  FAMILY|
|      886|    GAME|
|      662|   TOOLS|
+---------+--------+



In [89]:
def clean_installs(installs):
  return installs[:-1]

spark.udf.register("CLEAN_INSTALLS", clean_installs, StringType())
spark.sql("SELECT CLEAN_INSTALLS(Installs) FROM APPS").show()


+------------------------+
|CLEAN_INSTALLS(Installs)|
+------------------------+
|                     500|
|               1,000,000|
|                  10,000|
|                  10,000|
|                  10,000|
|                     100|
|                 100,000|
|                 500,000|
|                 100,000|
|                     100|
|                     500|
|              10,000,000|
|                   5,000|
|                  10,000|
|                  50,000|
|               1,000,000|
|              10,000,000|
|                  50,000|
|                       5|
|                   1,000|
+------------------------+
only showing top 20 rows

